# Choosing the right data file format for numerical computing

This notebook will go over the pros and cons of various data file formats common in numerical python workflows. It'll cover various concerns when storing data on-disk and how popular formats address these challenges; the what and why of these formats.

## Who am I?

* David Hoese <sub>pronounced like Haze</sub>
* Software Developer (Sr. Instrumentation Technologist)<br>
  Space Science and Engineering Center (SSEC)<br>
  University of Wisconsin - Madison
* Satpy and Vispy developer
* @djhoese on Twitter and GitHub

## Contents

TODO

## Why write data to disk?

1. State or Cache
   * a long running application needs to start where it left off
   * reuse calculation in future executions
   * user settings/preferences are saved for later use
2. Data Archival/Distribution/Sharing
   * results are shared with other people
   * results are shared with other software

## What kind of files are we working with?

* Plain text or binary file formats
* Primarily numeric data
* Optionally add custom metadata

## What are we **NOT** talking about?

* Databases
* Python's pickle format
* Storing user application settings
* **Custom** binary formats (no community/organization support)

## Why does file format matter?

There are many different formats that we can use in Python and we have
different ways to access them. Each one comes with various advantages and
disadvantages. When it comes to choosing a file format for a task we should
be concerned with a few key things:

* **Write speeds**<br>
  How long does it take to get data from it's in-memory format to disk?
* **Read speeds**<br>
  How long does it take to get data from disk to a usable in-memory format?
* **On-disk size**<br>
  How big are the files?
* **In-memory size**<br>
  How much memory does it take to open the file and get data out? Are there multiple copies of the data in memory?
* **Flexibility/Usefulness**<br>
  Can I do anything else with the format? Store metadata? Can I use it for this other use case? Can I lazily load data? Can I access data remotely? Can I compress the data? How much data can I store in one file?
* **Format Availability**<br>
  Is the format only readable by Python software? Do my users need to learn something new to read it? If the format isn't "built in" to a programming environment, can I easily install the necessary libraries?

## Warmup - Plain Text

Let's start with an example that uses plain text to write the last time that
the function was called. We first need to import a few modules and then
create our function to append to our data file called `last_time.txt`.

In [1]:
import os
import sys
from datetime import datetime, timedelta

DATA_FILE = 'last_time.txt'

def write_time():
    with open(DATA_FILE, 'a') as time_file:
        now = datetime.utcnow()
        data_line = "{:f}\n".format(now.timestamp())
        time_file.write(data_line)

In our function we get the current time in
[UTC](https://en.wikipedia.org/wiki/Coordinated_Universal_Time). We convert
it to epoch time (a.k.a. POSIX time), seconds since `1970-01-01 00:00:00`, and
write the string representation to the file as a single line.

Let's add a couple times to the file by running the `write_time` a couple
times below.

In [2]:
write_time()

And we can use the command `head` to print out the first couple lines of the file's contents:

In [5]:
!head $DATA_FILE

1568667832.827468
1568667832.827796
1568667832.827835
1568667832.827897
1568667832.827929
1568667832.827957
1568667832.827985
1568667832.828037
1568667832.828065
1568667832.828094


We've written data to the file, now let's read it back out. The below function
will read the content's of the file in to a list.

In [6]:
def read_times():
    exec_times = []
    with open(DATA_FILE, 'r') as time_file:
        for line in time_file:
            time_val = float(line.strip())
            exec_times.append(time_val)
    return exec_times

In [9]:
read_times()[:10]

[1568667832.827468,
 1568667832.827796,
 1568667832.827835,
 1568667832.827897,
 1568667832.827929,
 1568667832.827957,
 1568667832.827985,
 1568667832.828037,
 1568667832.828065,
 1568667832.828094]

The code above gives us a simple example of saving data from software to a
file on disk. We wrote a single value at a time and accumulate more
information as time went on. We were able to read these data back
in to python at a later time. Could we have done anything differently?

## Plain Text - Pros/Cons

Let's take the above concerns and look at our text file from before and the
code we used to access it.

<details>
<summary>Pros</summary>
    
* Human readable
* Simple code (no external libraries)
* Easily usable by other languages/tools
* Could read one value at a time (but we didn't)
    
</details>

<details>
    <summary>Cons</summary>

* Have to convert data to/from string and float (slow)
* Representing each 8 byte float (64-bit) as ~17 ASCII bytes
* Unknown precision of data values, how many decimal points?
* Don't know how many elements until it is fully read
* Can't easily seek to a specific index/element
* Code: Read as a list instead of a numpy array and used a python for loop (potentially slow)
    
</details>
<br>

And here's what a single value of our text file looks like on disk (8-bit ASCII character):

<table>
    <tr>
        <th>Byte Offset</th>
        <th>Value</th>
    </tr>
    <tr><td>0</td><td>1</td></tr>
    <tr><td>1</td><td>5</td></tr>
    <tr><td>2</td><td>6</td></tr>
    <tr><td>3</td><td>8</td></tr>
    <tr><td>4</td><td>6</td></tr>
    <tr><td>5</td><td>6</td></tr>
    <tr><td>6</td><td>7</td></tr>
    <tr><td>7</td><td>8</td></tr>
    <tr><td>8</td><td>3</td></tr>
    <tr><td>9</td><td>2</td></tr>
    <tr><td>10</td><td>.</td></tr>
    <tr><td>11</td><td>8</td></tr>
    <tr><td>12</td><td>2</td></tr>
    <tr><td>13</td><td>7</td></tr>
    <tr><td>14</td><td>4</td></tr>
    <tr><td>15</td><td>6</td></tr>
    <tr><td>16</td><td>8</td></tr>
    <tr><td>17</td><td>\n</td></tr>
    </table>

We can perform a quick timing to see how long it takes to read the file:

In [23]:
txt_time_write = %timeit -o -n 10000 -r 1 write_time()

16.8 µs ± 0 ns per loop (mean ± std. dev. of 1 run, 10000 loops each)


In [24]:
txt_time_read = %timeit -o -n 100 -r 1 read_times()

3.73 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 100 loops each)


The NumPy library also provides a function for loading data from text files.
Let's try it and see how it compares.

In [25]:
import numpy as np
txt_time_read_np = %timeit -o -n 100 -r 1 np.loadtxt(DATA_FILE)

19.1 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 100 loops each)


It seems that in this specific case and with all of the extra checks numpy
performs, it actually takes longer to read the data with numpy. When it comes
to simple human-readable formats, we couldn't have gone much simpler.

The remainder of this document will go through different use cases and
the file formats that we have as options. We'll apply this type of
performance analysis to our format choices.

## Flat Binary

When human-readability isn't necessary, another option for storing simple
data structures is a flat binary file. A flat binary file consists of the
raw data values stored contiguously as a flat array. Let's rewrite our code
from above to write to a flat binary file using the numpy library.

In [26]:
import numpy as np

BIN_DATA_FILE = 'last_time.dat'

def write_time_binary():
    with open(BIN_DATA_FILE, 'a') as time_file:
        now = datetime.utcnow()
        np.array([now.timestamp()], dtype=np.float64).tofile(time_file)

In [27]:
def read_times_binary():
     return np.fromfile(BIN_DATA_FILE, dtype=np.float64)

In [44]:
bin_time_write = %timeit -o -n 100000 -r 1 write_time_binary()

28.1 µs ± 0 ns per loop (mean ± std. dev. of 1 run, 100000 loops each)


In [45]:
bin_time_read = %timeit -o -n 100 -r 7 read_times_binary()

69.2 µs ± 13.7 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


## Perform Tip: Memory Maps

By default, when reading a file from disk we have to transfer data from
the hard disk to system memory. That means that when creating something
like a numpy array from a binary data file we are transferring **all** of the
file's contents from disk in to memory when we might not use it right away; a very slow operation. There is a
lazier, generally more efficient, method called memory mapping (a.k.a. mmap in
C, memmap by numpy). By creating a memory map, we allocate the virtual memory
space for our data, but the operating system won't load the data from disk
until we need it. Memory mapping avoids extra copies of file data in memory, works very well with random accesses to data in a file, can be cached more efficiently, shared between processes more effectively, and is generally your best option for reading large files that are difficult to hold in memory at a single time.

Further reading:

* [Stackoverflow answer discussing memory maps](https://stackoverflow.com/a/6383253/433202)
* [Memory-mapped File - Wikipedia](https://en.wikipedia.org/wiki/Memory-mapped_file)

Going back to the above binary file usage...

In [31]:
def read_times_binary_memmap():
    return np.memmap(BIN_DATA_FILE, mode='r', dtype=np.float64)

In [47]:
bin_time_read_mmap = %timeit -o -n 100 -r 7 read_times_binary_memmap()

32.4 µs ± 7.57 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [48]:
bin_time_read.average / bin_time_read_mmap.average

2.1368372537238214

Keep in mind that memory mapping isn't **loading** the data in to memory so this isn't technically a fair comparison. However, as we use the memory mapped array object we should see better performance than a traditional read of the file.

Note that we could also use memory maps to write data. This is most beneficial if we are writing to random locations in the file.

## Flat Binary - Pros/Cons

<details>
    <summary>Pros</summary>
    
* Simple code
* Readable by any programming language (*see Cons)
* Minimum on-disk size without compression
* Fast reading and memory mappable
* Supports N-dimensional data
* Subsettable

</details>
    
<details>
    <summary>Cons</summary>
    
* Not human readable
* No shape, data type, or byte order information stored
* Platform dependent (not shareable)

Note from numpy docs:
    
> Do not rely on the combination of tofile and fromfile for data storage, as the binary files generated are are not platform independent. In particular, no byte-order or data-type information is saved. Data can be stored in the platform independent .npy format using save and load instead.
    
</details>

Storage layout (64-bit float):

<table>
    <tr>
        <th>Byte Offset</th>
        <th>Value</th>
    </tr>
    <tr><td>0</td><td>1568667832.827468</td></tr>
    <tr><td>8</td><td>1568667832.827796</td></tr>
    <tr><td>16</td><td>1568667832.827835</td></tr>
    </table>

### Bonus - Numpy's .npy format

As mention in the cons above, a flat binary file *only* has the data and no
other information. This means we have to keep track of this information
ourselves. To help with this numpy provides a `.npy` format which stores this
information inside the file alongside the binary data. Here's a quick example
to create a `.npy` file using
[`np.save`](https://docs.scipy.org/doc/numpy/reference/generated/numpy.save.html#numpy.save).

In [56]:
np.save('last_time.npy', np.array([1., 2., 3.]))

When we are ready to read the data back we can use
[`np.load`](https://docs.scipy.org/doc/numpy/reference/generated/numpy.load.html#numpy.load).
This method gives us the option to open the data as a memory map, giving us
the space and speed advantages of a flat binary while avoiding the format
metadata issues. Keep in mind that this format is only readable by numpy and
would require an additional library in any other language.

In [57]:
np.load('last_time.npy', mmap_mode='r')

memmap([1., 2., 3.])

## Comma-Separated Values (CSV)

So far we've dealt with a single stream (a.k.a. field or variable) of data,
but what if we need to store more? When it comes to multiple 1-dimensional
variables one of the more common solutions is a Comma-Separate Values (CSV)
file. We could use numpy again, but instead we'll use the `pandas` library
for its more powerful handling of tabular data like we would store in a CSV.

We'll start by loading some example data used by the seaborn python package.
Their example data is stored as a
[CSV file on GitHub](https://raw.githubusercontent.com/mwaskom/seaborn-data/master/iris.csv).
We use the pandas
[`read_csv`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html)
function. This function has a lot of options, but we'll use the defaults for now.

In [58]:
import pandas as pd
import numpy as np

seaborn_iris_url = 'https://raw.githubusercontent.com/mwaskom/seaborn-data/master/iris.csv'
data = pd.read_csv(seaborn_iris_url)
print(data.shape)
data.head()

(150, 5)


,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


If we were making our CSV file from a pandas dataframe we can use `to_csv`:

In [59]:
my_df = pd.DataFrame(np.random.random((150, 5)), columns=['A', 'B', 'C', 'D', 'E'])
my_df.head()

,A,B,C,D,E
0,0.518009,0.625533,0.663213,0.752765,0.985216
1,0.176046,0.204976,0.907754,0.967893,0.398528
2,0.774760,0.296300,0.530271,0.627792,0.819883
3,0.614117,0.114465,0.713061,0.716801,0.926176
4,0.198582,0.099838,0.023641,0.082771,0.523411


In [60]:
my_df.to_csv('randoms.csv', index=False)

Pandas also provides options for memory mapping the text file to reduce I/O
overhead.

In [61]:
my_df = pd.read_csv('randoms.csv', memory_map=True)
my_df.head()

,A,B,C,D,E
0,0.518009,0.625533,0.663213,0.752765,0.985216
1,0.176046,0.204976,0.907754,0.967893,0.398528
2,0.774760,0.296300,0.530271,0.627792,0.819883
3,0.614117,0.114465,0.713061,0.716801,0.926176
4,0.198582,0.099838,0.023641,0.082771,0.523411


### Performance Tip: Chunking

So far we've been loading all data at once or depending on memory mapping to
reduce the amount of data that was loaded at any one time. Another possible
improvement can come from loading chunks of data at a time. This is similar
to what we did with the original plain text file iterating over the lines of
the file one at a time.

By chunking and iterating over the data we can process files that would not
fit in memory otherwise. For more info on chunking, see the
[pandas documentation](https://pandas.pydata.org/pandas-docs/stable/user_guide/io.html#iterating-through-files-chunk-by-chunk).

In [62]:
reader = pd.read_csv('randoms.csv', iterator=True, chunksize=4)
for idx, chunk_df in enumerate(reader):
    print(chunk_df)
    if idx >= 2: break

          A         B         C         D         E
0  0.518009  0.625533  0.663213  0.752765  0.985216
1  0.176046  0.204976  0.907754  0.967893  0.398528
2  0.774760  0.296300  0.530271  0.627792  0.819883
3  0.614117  0.114465  0.713061  0.716801  0.926176
          A         B         C         D         E
4  0.198582  0.099838  0.023641  0.082771  0.523411
5  0.451134  0.288878  0.599627  0.101376  0.254136
6  0.652017  0.752561  0.262403  0.144780  0.932307
7  0.910469  0.449665  0.533842  0.230622  0.112332
           A         B         C         D         E
8   0.588053  0.131510  0.093826  0.441200  0.665580
9   0.118112  0.783991  0.766641  0.014607  0.263812
10  0.453678  0.411469  0.183099  0.631480  0.020634
11  0.383354  0.166245  0.382711  0.222890  0.618206


This reduces the amount of memory being used at any one time while we work with a single chunk.

We can change how big of a chunk we get by calling `get_chunk` with the number of
rows to put in the DataFrame returned. Note how we are continuing our reading from the above cells since the reader is an iterator.

In [64]:
reader.get_chunk(6)

,A,B,C,D,E
18,0.443357,0.632036,0.785462,0.580356,0.150587
19,0.282351,0.008892,0.887258,0.058430,0.382341
20,0.526052,0.213249,0.617271,0.985686,0.174114
21,0.813454,0.314126,0.668044,0.504940,0.875045
22,0.521319,0.684906,0.575570,0.085062,0.889112
23,0.445894,0.196311,0.277273,0.387277,0.006089


## CSV - Pros/Cons

<details>
    <summary>Pros</summary>

* Human readable
* Can be read in Microsoft Excel (non-programmer collaboration)
* Lazy/iterable loading possible
* Row-based operations are relatively fast

</details>
    
<details>
    <summary>Cons</summary>

* Slow to read/write
* Wasted disk space
* Require reading all columns to get value for a single column
* Column-based operations are slow (see below)

</details>

An unfortunate storage layout for a 3 column CSV (8-bit ASCII characters):

<table>
    <tr>
        <th>Byte Offset</th>
        <th>Value</th>
        <th>Column</th>
    </tr>
    <tr><td>0</td><td>0.4433,</td><td>1</td></tr>
    <tr><td>7</td><td>0.623,</td><td>2</td></tr>
    <tr><td>13</td><td>setosa\n</td><td>3</td></tr>
    <tr><td>20</td><td>0.8866,</td><td>1</td></tr>
    <tr><td>27</td><td>0.31,</td><td>2</td></tr>
    <tr><td>32</td><td>virginica\n</td><td>3</td></tr>
    <tr><td>42</td><td>0.6644,</td><td>1</td></tr>
    </table>

The above table shows how a CSV file may be stored on disk. If we don't force all floating point numbers to have the same number of digits or string fields are not all the same size then we can't be sure how to quickly get all values for a single column (by calculating offsets) without reading every value for every column. This makes doing column-based calculations, like the average of an entire field, very slow and wasteful. If we are doing a calculation that requires all values in a single row, then this structure is fairly convenient; we can parse one row at a time efficiently.

### Further Reading

* [Dask DataFrames](https://docs.dask.org/en/latest/dataframe.html) for distributed and lazy pandas dataframes

## Review 1

* Store as text: Human-readable but slow
* Store as binary: Easily indexable and fast to read/write
* Memory maps: Better I/O in most cases
* Chunking: Good when working on one chunk at a time
* Flat binary: Simple, quick solution, multiple dimensions, no structure

## Parquet

Parquet is another tabular data format and can be thought of as the high
performance binary version of the CSV file.
Analytics workflows typically don't need to read every column from a tabular
format and storing data in something like a CSV file can be very wasteful.
The first difference in what Parquet brings to the table (pun intended) is storing data by
column (image right) instead of by row (image left).

<img src="https://www.kdnuggets.com/wp-content/uploads/dremio-table-1.jpg">
<center><sub>Credit: https://www.kdnuggets.com/2017/02/apache-arrow-parquet-columnar-data.html</sub></center>
<br>

If we keep the columns together it is easier to access one column more easily
and more quickly than data stored row by row.

## Performance Tip: Spatial and Temporal Locality

Modern CPUs have multiple levels of caching. The closer a cache level is to the CPU (the computation) the smaller it is and less it can store at any one time. These closer cache levels are also much faster to get data from. Conversely, caches that are further from the CPU are larger and slower to access. The diagram shows the various caching levels common in modern computers where L1 caches are the smallest and fastest, then L2, L3, main RAM memory, and finally the hard disk; the largest and slowest storage on the local machine. 

<img width="400px" src="https://softwarerajivprab.files.wordpress.com/2019/07/cache.png" alt="CPU L1 L2 L3 cache diagram">
<center><sub>Credit: https://software.rajivprab.com/2018/04/29/myths-programmers-believe-about-cpu-caches/</sub></center><br>

If we want the best performance out of the file format we are using then we want to do as many operations as possible with what is in the L1 cache before replacing it with new data. Otherwise, we could suffer from reloading the same data from slower caches. **Temporal locality** is the idea that if we access a particular memory location, we are very likely to access that same location again in the near future. **Spatial locality** is the idea that if we access a particular memory location, an operation in the near future is probably going to involve the data right next to it. Modern computers will assume this to be true and will predictively cache things to get the best performance. That means our best option is to use the memory the way the computer thinks we are going to use it.

## Performance Tip: Block sizes

Each storage device (cache, RAM, disk) will typically have a default size or "chunk" of data that it will operate on or store in one contiguous location or provide to another storage element.

Further Reading: [Wikipedia](https://en.wikipedia.org/wiki/Locality_of_reference)

## Performance Tip: Single Instruction, Multiple Data (SIMD)

In addition to the multiple cache levels, modern CPUs can also operate on multiple pieces of data at the same time with a single CPU instruction. These vectorized instructions are referred to as SIMD. The CPU is told to do one operation on many values (ex. add 5 to every value) and can perform it on multiple values at a time, in parallel. 

Even though SIMD instructions are low-level, NumPy does a lot of the work for us by telling the CPU to use SIMD instructions when possible. However, this usually depends on taking advantage of locality (see above) so that the CPU has all the values it is going to operate on.

Further Reading: [Wikipedia](https://en.wikipedia.org/wiki/SIMD)

<br>

Parquet's design for how data is stored on disk and operated on in-memory tries to take advantage of these concepts. Let's run through a basic parquet example to see how we can read and write a parquet file and how we can control some aspects of these complex topics. We'll start by reusing the seaborn sample data from before.

In [66]:
seaborn_iris_url = 'https://raw.githubusercontent.com/mwaskom/seaborn-data/master/iris.csv'
data = pd.read_csv(seaborn_iris_url)
print(data.shape)
data.head()

(150, 5)


,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


Let's write this data to a local parquet file using the `fastparquet` library (there are others):

In [68]:
import fastparquet as fp
fp.write('iris1.parq', data)

In [69]:
!ls -lh iris1.parq

-rw-rw-r-- 1 davidh davidh 8.4K Oct  7 21:18 iris1.parq


In [71]:
parq_file = fp.ParquetFile('iris1.parq')
parq_file.to_pandas().head()

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


By default fastparquet makes a single file with no compression and only stores entire columns together (no row-groups). Although these defaults may not provide the best performance for a particular use case, the format itself provides us some nice conviences. For example, since the data is stored by column we can quickly and efficiently load a limited set of the columns:

In [72]:
data2 = parq_file.to_pandas(['petal_length', 'petal_width'])
data2.head()

,petal_length,petal_width
0,1.4,0.2
1,1.4,0.2
2,1.3,0.2
3,1.5,0.2
4,1.4,0.2


## HDF5

## NetCDF4

## Zarr